# Install Required Software Environment

## Step 1: Create a `work_dir` context manager

This is useful when you want to run a command in a working directory and then automatically change back to your original directory 

In [ ]:
import os
from contextlib import contextmanager
from pathlib import Path

@contextmanager
def work_dir(new_dir):
    old_dir = os.getcwd()
    os.chdir(new_dir)
    try:
        yield
    finally:
        os.chdir(old_dir)

## Step 2: Install the `isce3` environment with Pixi

In [ ]:
with work_dir(Path.cwd().parent):
    !pixi install -e isce3

## Step 3: Register the `isce3` environment's Python kernel with `ipykernel`

In [ ]:
env_name = "isce3"
display_name = f'"{env_name} (Python)"'

!pixi run -e isce3 python -m ipykernel install \
  --user \
  --name $env_name \
  --display-name $display_name

## Step 4: Ensure notebook shell commands run in the `isce3` environment

This ensures that shell commands executed from inside a notebook with `!` run in the notebook kernel’s environment. This works by launching the entire Jupyter kernel process inside the Pixi environment, so the kernel’s PATH, environment variables, and Python executable all come from that Pixi environment, not from the parent JupyterLab environment.

In [ ]:
from jupyter_client.kernelspec import KernelSpecManager
import json

ksm = KernelSpecManager()
spec = ksm.get_kernel_spec(env_name)
kernel_dir = Path(spec.resource_dir)
kernel_json = kernel_dir / "kernel.json"

data = json.loads(kernel_json.read_text())
orig_argv = data.get("argv", [])

new_argv = [
    "pixi",
    "run",
    "--manifest-path",
    str(Path.cwd().parent),
    "-e",
    env_name,
] + orig_argv

data["argv"] = new_argv
kernel_json.write_text(json.dumps(data, indent=2))
print(f"Updated kernel.json at {kernel_json} with Pixi wrapper.")
print("argv:", data["argv"])

## Step 5: (Optional) Delete the environment and remove its `kernelspec`

In [ ]:
with work_dir(Path.cwd().parent):
    !pixi clean
    !jupyter kernelspec remove isce3 -y